In [8]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Run

In [17]:
import pandas as pd

runs = [
    "VAL_naive_doctaet-simple_zs-mistralaiMistral7BInstructv03-20240611-095123",
    "VAL_naive_doctaet-simple_fs-mistralaiMistral7BInstructv03-20240611-102154",
    "VAL_naive_doctaet-simple_zs-googlegemma7bit-20240611-130659",
    "VAL_naive_doctaet-simple_fs-googlegemma7bit-20240611-133609",
    "VAL_llama3_70b_fewshot_optimized02-20240531-105142",
    "VAL_llama3_8b_few_shot_template_initial-20240604-125136",
    "VAL_naive_doctaet-simple_zs-llama38b-20240611-212642",
    "VAL_naive_doctaet-simple_fs-llama38b-20240611-213146",
    "VAL_naive_doctaet-simple_zs-gemma7b-20240612-073229",
    "VAL_naive_doctaet-simple_fs-gemma7b-20240612-073907",
    "VAL_naive_doctaet-simple_zs-mistral7b-20240612-074427",
    "VAL_naive_doctaet-simple_fs-mistral7b-20240612-075256"
]

# Top runs on Test:
# - TEST_llama3_70b_fewshot_initial-20240601-211554
dfs = []

for run in runs:
    path = f"results/{run}/df.feather"

    df = pd.read_feather(path)
    try:
        df = df.rename({"annontation": "annotation"}, axis=1) # typo in early versions
    except:
        pass
    dfs.append(df)


# Post Processing

In [18]:
from src.content_extraction import format

# Some older runs might not be in the right format -> format function is idempotent
for df in dfs:
    df["annotation"] = df["annotation"].apply(format)
    df["annotation"] = df["annotation"].astype(pd.StringDtype())
    df["ground_truth"] = df["ground_truth"].astype(pd.StringDtype())

In [19]:
df.dtypes

run             object
f               object
annotation      string
ground_truth    string
dtype: object

# Evaluation

In [20]:
# evaluate
from scoring_program.evaluation import Metrics

results = []
for df in dfs:
    result = Metrics.evaluate_property_wise_json_based(
        label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"])
    )

    result.update(
        Metrics.evaluate_rouge(label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"]))
    )
    result["run"] = df["run"][0]
    result["score"] = result["partial_f1s_overall"]
    result["num_samples"] = len(df)
    results.append(pd.DataFrame([result]))

df_res = pd.concat(results)
df_res

,general_accuracy,exact_recalls_task,exact_recalls_dataset,exact_recalls_metric,exact_recalls_Score,exact_recalls_overall,partial_recalls_task,partial_recalls_dataset,partial_recalls_metric,partial_recalls_Score,...,partial_f1s_metric,partial_f1s_Score,partial_f1s_overall,rouge1,rouge2,rougeL,rougeLsum,run,score,num_samples
0,51.0,0.99,0.99,4.93,0.99,1.97,6.40,0.99,4.93,0.97,...,5.35,1.50,3.81,18.73,11.54,17.80,17.47,VAL_naive_doctaet-simple_zs-mistralaiMistral7B...,3.81,100
0,64.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,34.51,14.45,33.73,33.71,VAL_naive_doctaet-simple_fs-mistralaiMistral7B...,0.00,100
0,51.0,0.00,0.00,0.00,0.00,0.00,0.49,0.00,0.00,0.00,...,0.00,0.00,0.15,13.28,2.28,11.40,11.30,VAL_naive_doctaet-simple_zs-googlegemma7bit-20...,0.15,100
0,58.0,0.00,0.00,0.00,0.00,0.00,0.49,0.00,0.49,0.00,...,0.98,0.00,0.49,16.87,1.52,16.16,16.11,VAL_naive_doctaet-simple_fs-googlegemma7bit-20...,0.49,100
0,49.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,49.00,0.00,49.00,49.00,VAL_llama3_70b_fewshot_optimized02-20240531-10...,0.00,100
0,69.0,4.93,13.79,7.39,1.97,7.02,10.84,14.78,8.87,1.94,...,7.16,1.43,6.91,34.03,7.58,33.12,33.04,VAL_llama3_8b_few_shot_template_initial-202406...,6.91,100
0,61.0,0.99,1.97,0.49,0.00,0.86,2.46,1.97,1.48,0.97,...,2.50,1.75,2.87,22.95,4.26,21.58,21.80,VAL_naive_doctaet-simple_zs-llama38b-20240611-...,2.87,100
0,62.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,22.40,3.54,21.23,21.29,VAL_naive_doctaet-simple_fs-llama38b-20240611-...,0.00,100
0,52.0,0.00,1.48,0.49,0.00,0.49,1.48,1.48,1.48,0.49,...,2.61,0.94,2.23,6.36,1.70,6.01,5.90,VAL_naive_doctaet-simple_zs-gemma7b-20240612-0...,2.23,100
0,55.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,12.04,0.85,11.72,11.46,VAL_naive_doctaet-simple_fs-gemma7b-20240612-0...,0.00,100


In [21]:
dfs[-1]

,run,f,annotation,ground_truth
0,VAL_naive_doctaet-simple_fs-mistral7b-20240612...,1503.05062,[{'LEADERBOARD': {'Task': 'Illumination of Mon...,unanswerable
1,VAL_naive_doctaet-simple_fs-mistral7b-20240612...,1109.0784,[{'LEADERBOARD': {'Task': 'Implementing Sharin...,unanswerable
2,VAL_naive_doctaet-simple_fs-mistral7b-20240612...,2210.16422v1,"[{'model': '5.21', 'score': None, 'type': 'Bas...",[{'LEADERBOARD': {'Task': 'Text Summarization'...
3,VAL_naive_doctaet-simple_fs-mistral7b-20240612...,2310.07488v2,"[{'model': 'Open Assistant', 'dataset_en': 'GS...",[{'LEADERBOARD': {'Task': 'Arithmetic Reasonin...
4,VAL_naive_doctaet-simple_fs-mistral7b-20240612...,2210.15425v1,"[ { 'name': '\prjname', 'paper_year': '', 'met...","[{'LEADERBOARD': {'Task': 'Keyword Spotting', ..."
...,...,...,...,...
95,VAL_naive_doctaet-simple_fs-mistral7b-20240612...,2006.09264v3,[{'LEADERBOARD': {'Task': 'One-Shot Neural Arc...,[{'LEADERBOARD': {'Task': 'Neural Architecture...
96,VAL_naive_doctaet-simple_fs-mistral7b-20240612...,1304.2639,[{'LEADERBOARD': {'Task': 'Reachability Proble...,unanswerable
97,VAL_naive_doctaet-simple_fs-mistral7b-20240612...,1803.10683v3,"[ { 'name': 'Training Method', 'values': ['GT ...","[{'LEADERBOARD': {'Task': 'Pose Estimation', '..."
98,VAL_naive_doctaet-simple_fs-mistral7b-20240612...,2006.06936v2,"[{'method': 'Random Search', 'queries': 1000, ...",[{'LEADERBOARD': {'Task': 'Neural Architecture...


In [22]:
format("[]")


'unanswerable\n'